# Data prepration in python

# Data reading

In [1]:
import glob
import gensim
import numpy as np

In [2]:
ff=glob.glob("/Users/atousa/Desktop/McMaster/BDA-102/Week4-TextMining/SportsArticles/Raw_data/*.txt")

In [3]:
len(ff)

1000

In [13]:
ff[0]

'/Users/atousa/Desktop/McMaster/BDA-102/Week4-TextMining/SportsArticles/Raw_data/Text0001.txt'

In [4]:
raw_doc=[]
for file in ff:
    try:
        f=open(file,"r")
        raw_doc.append(f.read())
    except:
        pass

In [5]:
len(raw_doc)

866

In [6]:
clean_texts=[]
for text in raw_doc:
    #This lowercases, tokenizes, 
    clean_texts.append(gensim.utils.simple_preprocess(text))

In [7]:
clean_texts[0][:10]

['finalists',
 'in',
 'the',
 'apertura',
 'play',
 'offs',
 'toluca',
 'had',
 'drawn',
 'their']

In [8]:
Dictionary=gensim.corpora.Dictionary(clean_texts)
print(len(Dictionary))

22771


In [42]:
for i in range(10):
    print(i,Dictionary[i])

0 against
1 ahead
2 aldo
3 all
4 also
5 apertura
6 arizala
7 benitez
8 britos
9 but


In [11]:
Dictionary.token2id['apertura']

5

In [12]:
import numpy as np
np.array(clean_texts[0]).T

array(['finalists', 'in', 'the', 'apertura', 'play', 'offs', 'toluca',
       'had', 'drawn', 'their', 'first', 'two', 'clausura', 'games',
       'but', 'got', 'off', 'to', 'good', 'start', 'when', 'edgar',
       'benitez', 'put', 'them', 'ahead', 'in', 'the', 'th', 'minute',
       'matias', 'britos', 'levelled', 'minutes', 'later', 'but', 'lucas',
       'silva', 'netted', 'minutes', 'from', 'the', 'end', 'to', 'ensure',
       'the', 'visitors', 'took', 'all', 'three', 'points', 'franco',
       'arizala', 'scored', 'minutes', 'from', 'time', 'to', 'ensure',
       'jaguares', 'claimed', 'their', 'first', 'point', 'with', 'draw',
       'against', 'monterrey', 'who', 'had', 'opened', 'the', 'scoring',
       'through', 'aldo', 'de', 'nigris', 'hosts', 'jaguares', 'also',
       'had', 'jorge', 'rodriguez', 'sent', 'off', 'in', 'the', 'closing',
       'moments'], dtype='<U9')

Now we will create a corpus. A corpus is a list of bags of words. A bag-of-words representation for a document just lists the number of times each word occurs in the document.

In [37]:
corpus=[Dictionary.doc2bow(text) for text in clean_texts]
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 2), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 3), (27, 1), (28, 3), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 6), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 3), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1)]


In [38]:
tf_idf=gensim.models.TfidfModel(corpus)
print(tf_idf)

TfidfModel(num_docs=866, num_nnz=248622)


In [61]:
ll=np.argsort(np.array([w[1] for w in tf_idf[corpus[0]]]))

In [63]:
np.array([Dictionary[w] for w in ll]).T

array(['the', 'to', 'in', 'with', 'who', 'but', 'when', 'all', 'two',
       'time', 'from', 'also', 'three', 'play', 'good', 'against',
       'first', 'them', 'games', 'got', 'their', 'end', 'points', 'had',
       'point', 'through', 'put', 'took', 'start', 'th', 'off', 'scored',
       'later', 'ahead', 'minute', 'scoring', 'draw', 'sent', 'opened',
       'de', 'moments', 'claimed', 'visitors', 'drawn', 'closing',
       'benitez', 'hosts', 'silva', 'minutes', 'rodriguez', 'lucas',
       'netted', 'edgar', 'offs', 'jorge', 'monterrey', 'levelled',
       'finalists', 'aldo', 'toluca', 'nigris', 'clausura', 'franco',
       'matias', 'britos', 'arizala', 'apertura', 'ensure', 'jaguares'],
      dtype='<U9')

In [74]:
similarity_object=gensim.similarities.Similarity('/Users/atousa/Desktop/McMaster/BDA-102/data/SportsArticles/Raw_data/',tf_idf[corpus],num_features=len(Dictionary))

In [77]:
similarity_object.num_best = 3
similarity_object[tf_idf[corpus[0]]]

[(0, 1.0000001192092896), (783, 0.2790677547454834), (13, 0.15426906943321228)]

Now create a query document and convert it to tf-idf.

In [93]:
text=raw_doc[8]
query_doc=gensim.utils.simple_preprocess(text)
query_doc_bow=Dictionary.doc2bow(query_doc)
query_doc_tf_idf=tf_idf[query_doc_bow]

In [94]:
similarity_scores=list(similarity_object[query_doc_tf_idf])

In [97]:
max_score=max(similarity_scores)

In [98]:
print(max_score)

0.9999997


In [99]:
similarity_scores.index(max_score)

8

In [100]:
sorted_score=sorted(similarity_scores,reverse=True)
print(sorted_score[0])
print(sorted_score[1])

0.9999997
0.18975124


In [101]:
similarity_scores.index(sorted_score[1])

40

In [102]:
raw_doc[40]

'Print\nEurosport - Ben Foster earned five England caps before retiring from the international game\nFoster has been in prime form for Albion during his 18 months at the Hawthorns and feels his performance in the 2-0 win at Liverpool last week was the best of his career.\nBut he has never wavered from his decision to stop making himself available for his country just under two years ago when a Birmingham player. Clarke said: "If you\'re looking at a top-line goalkeeper, every mistake is analysed. If you go with England, it\'s magnified 10 times."\nHe continued: "Ben decided he\'d rather spend time with his family than be put under that stress and strain and we respect that. For us as a club, it\'s good.\n"Ben likes to be out of the limelight. He likes to go out, do his training and play as well as he can for Albion.\n"He gets enough satisfaction from that to keep his career going forward."\nClarke also assured midfielder Zoltan Gera, currently recovering from a cruciate ligament injury

# Text Summerization

In [54]:
from gensim.summarization import summarize
from gensim.summarization import keywords

In [57]:
print(summarize(raw_doc[40],word_count=100))

Eurosport - Ben Foster earned five England caps before retiring from the international game
Foster has been in prime form for Albion during his 18 months at the Hawthorns and feels his performance in the 2-0 win at Liverpool last week was the best of his career.
Gera, who is set to return for pre-season training in July, is out of contract in the summer although Albion have a 12 month option in their favour.
"If he's with the squad during the first week of pre-season training, then we can start thinking about the contract."
